In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-Wine-Quality-predidection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-End-Wine-Quality-predidection'

In [5]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
#Configuration Manager

from ML_Project.constants import *
from ML_Project.entity.config_entity import DataIngestionConfig, DataTransformationConfig, DataValidationConfig
from ML_Project.utils.common import create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath= SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([Path(self.config.artifact_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
    
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            artifact_dir=self.config.artifact_root
        )
       
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        root_dir=Path(config.root_dir)
        create_directories([root_dir])
        data_validation_config=DataValidationConfig(
            root_dir=Path(config.root_dir),
            unzip_data_dir=Path(config.unzip_data_dir),
            STATUS_FILE=Path(config.STATUS_FILE),
            all_schema=self.schema
        )
        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation
        root_dir=Path(config.root_dir)
        create_directories([root_dir])
        data_transformation_config=DataTransformationConfig(
            root_dir=root_dir,
            data_path=Path(config.data_path)
        )
        return data_transformation_config
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.Elasticsearch
        schema=self.schema.TARGET_COLUMN
        model_trainer_config=ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config











In [7]:

import joblib
from sklearn.linear_model import ElasticNet
import pandas as pd
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        X_train=train_data.drop(columns=[self.config.target_column],axis=1)
        y_train=train_data[self.config.target_column]
        model=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio)
        model.fit(X_train,y_train) 
        os.makedirs(self.config.root_dir, exist_ok=True)
        joblib.dump(model,os.path.join(self.config.root_dir,self.config.model_name))
     


In [8]:

config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer=ModelTrainer(config=model_trainer_config)
model_trainer.train()


[2025-10-07 17:01:23,795] :INFO:common: Created directory at: artifacts :
